<a href="https://colab.research.google.com/github/luisosmx/Project_Technical_Challenge/blob/main/Technical__challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Common imports
import numpy as np
import os

# Parquet imports
import pyarrow as pa
import pyarrow.parquet as pq

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


import pandas as pd
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-06-02.tar.gz

--2023-08-16 18:01:47--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-06-02.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.218.192.209, 52.92.179.50, 3.5.82.11, ...
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.218.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18657952487 (17G) [binary/octet-stream]
Saving to: ‘cord-19_2022-06-02.tar.gz’

cord-19_2022-06-02. 100%[===================>]  17.38G  26.5MB/s    in 12m 37s 

2023-08-16 18:14:25 (23.5 MB/s) - ‘cord-19_2022-06-02.tar.gz’ saved [18657952487/18657952487]



In [6]:
import tarfile


tar = tarfile.open('cord-19_2022-06-02.tar.gz', 'r:gz')
tar.extractall()  # specify path if necessary
tar.close()

In [7]:
df = pd.read_csv('/content/2022-06-02')
df.head(10)

IsADirectoryError: ignored